## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from time import time, sleep
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from itertools import chain

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 100
NUMBER_OF_ACTIVATED_PARTICIPANTS = 20
INITIAL_BALANCE = str(110_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)
CLAIM_AMOUNT_LIST = [int(round(20_000 * random.random(), -3)) + 20_000 for _ in range(NUMBER_OF_PARTICIPANTS)]

NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
PREFIXES = ['cosmos', 'osmo', 'terra']

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(3)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(2)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf'

INIT_TREASURY_CONTRACT = True
TREASURY_CODE_ID = str(4)
TREASURY_CONTRACT_ADDRESS = 'bostrom18cszlvm6pze0x9sz32qnjq4vtd45xehqs8dq7cwy8yhq35wfnn3q9u647v'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(1)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 100
Number of threads: 100


  0%|          | 0/100 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,39000,john16524446570,QmPoZoWScNbRC1cqMDfwFk4srgJWajp4LUnzpZnCQq1G73,0x77733286f58e09dd27229a4c43aa8c589c055b09,0xda59de6877336a18c1ddb0bcce1c9e12ba6e5ce7dc6c...,bostrom1xmdpqy8mlwnud5whzgsva38tws29qwhluftw4x,cosmos1xmdpqy8mlwnud5whzgsva38tws29qwhll6latp,fiber awkward industry race design bird transf...,bostrom1xmdpqy8mlwnud5whzgsva38tws29qwhluftw4x...,0x30d7ee1f90e75cbb42180b675658190f7afce4846157...,{'pub_key': 'AqFUzM7XjyUgIzkaJ3xsNiM33/hhI2A2K...,eyJwdWJfa2V5IjoiQXFGVXpNN1hqeVVnSXprYUozeHNOaU...
1,22000,john16524446571,QmU2wN9M9ue2mTvZ1e6EGw8prtQjUTfkuR7cKCvMPLSZuY,0x38c94ad36f6d99affde350f6d458dd851be95c88,0x03e5f19ded2a4b31fecb75e9a57abe40d0bdd6edf484...,bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v,cosmos1shgxepz7m5dycfmd0gd8vxxvp8uelx6p96j63t,wink box during grief liquid maid faint donate...,bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v...,0x4f84ef6848c70cddb5e66ff78038c3f030fadb08cbbd...,{'pub_key': 'AhsicgXQ9qnBrDIpMdCC+ft3GsqGnne+/...,eyJwdWJfa2V5IjoiQWhzaWNnWFE5cW5CckRJcE1kQ0MrZn...
2,28000,john16524446572,QmP1PnnK1EWzD5VheBrdPBKydRZdvqrVWLXhTak1WpjuMw,0xafcfc121796b6c1ee89e432a15793f2efa7e130c,0x7a6ae4502055818a38cb4f19103f77f65036904ff609...,bostrom19ssajxkwu8kvff9um8nmhr4kfnptpp8vevrzw4,cosmos19ssajxkwu8kvff9um8nmhr4kfnptpp8v6lh3sj,half famous helmet oil expand gold barely stor...,bostrom19ssajxkwu8kvff9um8nmhr4kfnptpp8vevrzw4...,0x876a2ee206f67f0898686ed7e3768aec1d4ab72ed47f...,{'pub_key': 'A6ZRALY1N25cUS2bhLSTG+Bd703Zev3qe...,eyJwdWJfa2V5IjoiQTZaUkFMWTFOMjVjVVMyYmhMU1RHK0...
3,27000,john16524446573,QmWkjPj9Rf5RtJcAm2EFL1iWTxRGWHYxhszxHckRKHDcqe,0xffe9bee97bf5b4cfde050cbd7e4cfefec41f37fa,0xe5c340540f74cc190afc2a5bc37038bab1c3c7f32e2a...,bostrom1eqdfdkllfpcnlcwrpppuq8v0z3j20aqereh3v4,cosmos1eqdfdkllfpcnlcwrpppuq8v0z3j20aqeq2rzjj,hold predict empower display clog panic entire...,bostrom1eqdfdkllfpcnlcwrpppuq8v0z3j20aqereh3v4...,0x5e7608320377526131ef41a46c4a71c6e5df98d874e3...,{'pub_key': 'AlFr40raiXo1Q/E20X193lGDzRc1xfeT3...,eyJwdWJfa2V5IjoiQWxGcjQwcmFpWG8xUS9FMjBYMTkzbE...
4,37000,john16524446574,QmV2AUTLGbzRdhFNgCaM9eaucRUSpSF7NuzKoJCvcZdJh2,0xeabaee4f86d50c6fbb4f5925e649c864d684d6cc,0x7a0bc17c08f9e25771a2016b7c6ca562a84306fd6d90...,bostrom1f7s9vzv79ewa6htzd0gzhrdxf9p45luxvjtjhv,osmo1f7s9vzv79ewa6htzd0gzhrdxf9p45lux86v3le,strong air mosquito police behind pear warfare...,bostrom1f7s9vzv79ewa6htzd0gzhrdxf9p45luxvjtjhv...,0x92fc66a16e33c62edf7aefcf0f934889501b3bb355ec...,{'pub_key': 'AwR4kuj4TZnKMvDbFEO6Tgv/CRW5eriKp...,eyJwdWJfa2V5IjoiQXdSNGt1ajRUWm5LTXZEYkZFTzZUZ3...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list_ethereum = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['ethereum_address', 'amount']].values.tolist()]
root_source_list_cosmos = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['cosmos_address', 'amount']].values.tolist()]
root_source_list = list(chain.from_iterable(zip(root_source_list_ethereum, root_source_list_cosmos)))

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 30
BASH_SIZE = 2000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_ACTIVATED_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_ACTIVATED_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS:>,}\nNumber of activated participants: {NUMBER_OF_ACTIVATED_PARTICIPANTS:>,}\nNumber of threads: {NUMBER_OF_THREADS:>,}\nBash size: {BASH_SIZE:>,}\nNumber of tasks: {len(tasks):>,}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

Number of addresses: 200
Number of activated participants: 20
Number of threads: 30
Bash size: 2,000
Number of tasks: 1


  0%|          | 0/1 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 41
Merkle root: 262b663b1158c081791de9d0a24c09dd2129fba74e28c4efe8d03928d64fb437
Number of addresses in the Merkle tree: 41
Done in 1.18s.



In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='inner',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='inner',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,39000,john16524446570,QmPoZoWScNbRC1cqMDfwFk4srgJWajp4LUnzpZnCQq1G73,0x77733286f58e09dd27229a4c43aa8c589c055b09,0xda59de6877336a18c1ddb0bcce1c9e12ba6e5ce7dc6c...,bostrom1xmdpqy8mlwnud5whzgsva38tws29qwhluftw4x,cosmos1xmdpqy8mlwnud5whzgsva38tws29qwhll6latp,fiber awkward industry race design bird transf...,bostrom1xmdpqy8mlwnud5whzgsva38tws29qwhluftw4x...,0x30d7ee1f90e75cbb42180b675658190f7afce4846157...,{'pub_key': 'AqFUzM7XjyUgIzkaJ3xsNiM33/hhI2A2K...,eyJwdWJfa2V5IjoiQXFGVXpNN1hqeVVnSXprYUozeHNOaU...,[a53ec1fe56e5d811717a6094a58a7de639a68f75bfe0e...,[3f9ba271dd2fd7f4c8efeb867fd038e5d3730fc2497e5...
1,22000,john16524446571,QmU2wN9M9ue2mTvZ1e6EGw8prtQjUTfkuR7cKCvMPLSZuY,0x38c94ad36f6d99affde350f6d458dd851be95c88,0x03e5f19ded2a4b31fecb75e9a57abe40d0bdd6edf484...,bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v,cosmos1shgxepz7m5dycfmd0gd8vxxvp8uelx6p96j63t,wink box during grief liquid maid faint donate...,bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v...,0x4f84ef6848c70cddb5e66ff78038c3f030fadb08cbbd...,{'pub_key': 'AhsicgXQ9qnBrDIpMdCC+ft3GsqGnne+/...,eyJwdWJfa2V5IjoiQWhzaWNnWFE5cW5CckRJcE1kQ0MrZn...,[12a8de125cbf3ef5b253dd22dd48b2e47d909245942ab...,[11134c99cc32ad3d35894f454196872926bed1b983a82...
2,28000,john16524446572,QmP1PnnK1EWzD5VheBrdPBKydRZdvqrVWLXhTak1WpjuMw,0xafcfc121796b6c1ee89e432a15793f2efa7e130c,0x7a6ae4502055818a38cb4f19103f77f65036904ff609...,bostrom19ssajxkwu8kvff9um8nmhr4kfnptpp8vevrzw4,cosmos19ssajxkwu8kvff9um8nmhr4kfnptpp8v6lh3sj,half famous helmet oil expand gold barely stor...,bostrom19ssajxkwu8kvff9um8nmhr4kfnptpp8vevrzw4...,0x876a2ee206f67f0898686ed7e3768aec1d4ab72ed47f...,{'pub_key': 'A6ZRALY1N25cUS2bhLSTG+Bd703Zev3qe...,eyJwdWJfa2V5IjoiQTZaUkFMWTFOMjVjVVMyYmhMU1RHK0...,[f134ebdd5b30a07f715cc1d3a4aec8c8f6afb2bf0584e...,[128880e27967800ebc916608fc66df0c9c3108f448be5...
3,27000,john16524446573,QmWkjPj9Rf5RtJcAm2EFL1iWTxRGWHYxhszxHckRKHDcqe,0xffe9bee97bf5b4cfde050cbd7e4cfefec41f37fa,0xe5c340540f74cc190afc2a5bc37038bab1c3c7f32e2a...,bostrom1eqdfdkllfpcnlcwrpppuq8v0z3j20aqereh3v4,cosmos1eqdfdkllfpcnlcwrpppuq8v0z3j20aqeq2rzjj,hold predict empower display clog panic entire...,bostrom1eqdfdkllfpcnlcwrpppuq8v0z3j20aqereh3v4...,0x5e7608320377526131ef41a46c4a71c6e5df98d874e3...,{'pub_key': 'AlFr40raiXo1Q/E20X193lGDzRc1xfeT3...,eyJwdWJfa2V5IjoiQWxGcjQwcmFpWG8xUS9FMjBYMTkzbE...,[da3ee3cbb3abec0f903a5d35e01430612b79426f52f72...,[9146ef5d2c6679e6711cdd4c4245428e10d09914dc3e3...
4,37000,john16524446574,QmV2AUTLGbzRdhFNgCaM9eaucRUSpSF7NuzKoJCvcZdJh2,0xeabaee4f86d50c6fbb4f5925e649c864d684d6cc,0x7a0bc17c08f9e25771a2016b7c6ca562a84306fd6d90...,bostrom1f7s9vzv79ewa6htzd0gzhrdxf9p45luxvjtjhv,osmo1f7s9vzv79ewa6htzd0gzhrdxf9p45lux86v3le,strong air mosquito police behind pear warfare...,bostrom1f7s9vzv79ewa6htzd0gzhrdxf9p45luxvjtjhv...,0x92fc66a16e33c62edf7aefcf0f934889501b3bb355ec...,{'pub_key': 'AwR4kuj4TZnKMvDbFEO6Tgv/CRW5eriKp...,eyJwdWJfa2V5IjoiQXdSNGt1ajRUWm5LTXZEYkZFTzZUZ3...,[058db2378e06ddb5bda7d217bec41c356d11e1fff2161...,[652e447ac6f4d4d409b84bc3ace2103d692cb3ab78a1c...


In [6]:
for i in range(ceil(NUMBER_OF_ACTIVATED_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza
Avatar subgraph contract address: bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf
Proof subgraph contract address: bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [10]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom199j0396e5rsdjp49v492l75u82xtgsuxl74jswawvefjggpj5ctsua2nl6


### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1pv56ghmgnt9sv0ry6fg3rqf6wgfnckksppltamwsdnd0e0e0kfms92ntpq


### Initiate Class of Output Parsing

In [12]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [13]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 110000000boot

coin spent
	spender: Passport Owner Address
	amount: 110000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 110000000boot
Gas used: 73,150
Tx hash: 355AD13B48082E233292A4E767EF8C5307ABA054C5D14272FFEF3821888479C0


In [14]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 110,000,000
Gas used: 135,284
Tx hash: 90612CA9717CD059FC09821D723DEF4BEB8C2FE234B99B1A4C75B50185903558


### Register Merkle Root

In [15]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 262b663b1158c081791de9d0a24c09dd2129fba74e28c4efe8d03928d64fb437
Gas used: 131,251
Tx hash: AC0A3CDBDED5ACB837ED3533EB13E6A9F08BFBC9B4B58A35B2164C1C9AAFF08C


#### Get Merkle Root form the Gift Contract

In [16]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1pv56ghmgnt9sv0ry6fg3rqf6wgfnckksppltamwsdnd0e0e0kfms92ntpq
{'data': {'merkle_root': '262b663b1158c081791de9d0a24c09dd2129fba74e28c4efe8d03928d64fb437'}}


### Send coins to new addresses

In [17]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 900

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

send_output = []
for i, bostrom_addresses_item in tqdm(enumerate(bostrom_addresses_chunks)):
    try:
        send_output.append(
            contract_utils.send_coins(
                from_seed=WALLET_SEED,
                to_addresses=bostrom_addresses_item,
                amounts=[1] * len(bostrom_addresses_item),
                gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(24e6)),
                display_data=DISPLAY_TX_EXECUTION))
    except Exception as e:
        print(f'Error in the {i} chunk: {e}')
        sleep(15)

0it [00:00, ?it/s]

In [18]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output[0])


Events

coin received
	receiver: bostrom1xmdpqy8mlwnud5whzgsva38tws29qwhluftw4x
	amount: 1boot
	receiver: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	amount: 1boot
	receiver: bostrom19ssajxkwu8kvff9um8nmhr4kfnptpp8vevrzw4
	amount: 1boot
	receiver: bostrom1eqdfdkllfpcnlcwrpppuq8v0z3j20aqereh3v4
	amount: 1boot
	receiver: bostrom1f7s9vzv79ewa6htzd0gzhrdxf9p45luxvjtjhv
	amount: 1boot
	receiver: bostrom1lefp47zz6r4tzmy7f6sr4ns85jtj7k6gyq82p4
	amount: 1boot
	receiver: bostrom14x74gw7ukhyntwgp46dzz5gjv022hra4pthm74
	amount: 1boot
	receiver: bostrom1fmszqsz52cg4ez05du8lxmrllhzmgfehdn4v5f
	amount: 1boot
	receiver: bostrom1mplmd4h768z98ftzztwdrldswvws3xg6jzv524
	amount: 1boot
	receiver: bostrom16xhjx3v840m8p466pzq6n7zja67c4vggrvvk5g
	amount: 1boot
	receiver: bostrom1lvnud0sdnavg5mp9p2rvnndqvmlae4n37tmz7u
	amount: 1boot
	receiver: bostrom1xp30e4lrl3775wa7e3zqyls33q3xc3vtnt3kjr
	amount: 1boot
	receiver: bostrom1jd6jjaxhxapw0gsva6s5pnelyw6u330t3fzkrp
	amount: 1boot
	receiver: bostrom1ez2xefskr

### Create Passports, Proof Addresses and Claim Gift

In [19]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 20
Number of threads: 100


  0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
with open('temp/contract_participation_execution_log_10000.txt', 'r') as f:
    data = json.load(f)

In [21]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	owner: bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n
	token_id: 518458
Gas used: 342,373
Tx hash: 5828264EA6AB3B89BFDDB4BDD9F93B97D3AB52DF0C7715712C73044D3864B339


In [22]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16524182639991
	address: 0x18fb581ee5d55bdbb4628490fcc323a8683e0182
Gas used: 237,523
Tx hash: 2487472AC6E27B38D36E7488464426BD4E606C06CF6393D2E3EE072A8C441043


In [23]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16524182639991
	address: cosmos18x96qyxfj9kxnqerj5ye4430gl4mdm0032vlm5
Gas used: 240,955
Tx hash: 17522237555D106CABA1FB0E454803AB674B8C7A408CC6BC36907254841DE6ED


In [24]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n
	amount: 100000boot

coin spent
	spender: bostrom1nfktgkcdysf38rx6wcycfwz5yhuw6cq9el970p845sj9nm5c99sq6qs023
	amount: 100000boot

execute
	execute contract: bostrom1zvjnc08uy0zz43m0nlh9f5aetpa3amn6a034yqvmsgvzshk9cldsqf2n5c
	execute contract: bostrom1nfktgkcdysf38rx6wcycfwz5yhuw6cq9el970p845sj9nm5c99sq6qs023

message from bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n
	sender: bostrom1nfktgkcdysf38rx6wcycfwz5yhuw6cq9el970p845sj9nm5c99sq6qs023
	amount: 100000boot

wasm
	_contract_address: bostrom1zvjnc08uy0zz43m0nlh9f5aetpa3amn6a034yqvmsgvzshk9cldsqf2n5c
	action: claim
	original: cosmos18x96qyxfj9kxnqerj5ye4430gl4mdm0032vlm5
	target: bostrom18x96qyxfj9kxnqerj5ye4430gl4mdm00jecv9n
	amount: 264,200
	_contract_address: bostrom1nfktgkcdysf38rx6wcycfwz5yhuw6cq9el970p845sj9nm5c99sq6qs023
	action: execute

### Release Gift

In [25]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 20
Number of threads: 100


  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [27]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	amount: 18336boot

coin spent
	spender: Treasury Contract
	amount: 18336boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	sender: Treasury Contract
	amount: 18336boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	gift_address: 0x38c94ad36f6d99affde350f6d458dd851be95c88
	stage: 9
	amount: 18,336
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,301
Tx hash: 6493B9D0BBF3FF00D69730BFD1144C990EBCCF9341970125DD8C15D4E98831A0


In [28]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	amount: 17587boot

coin spent
	spender: Treasury Contract
	amount: 17587boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	sender: Treasury Contract
	amount: 17587boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	gift_address: cosmos1shgxepz7m5dycfmd0gd8vxxvp8uelx6p96j63t
	stage: 9
	amount: 17,587
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,516
Tx hash: B502CF29760065754C45C0F70B3B076667432BF17B0A95FFECC115567F0AB074


## Passport NFT testing

In [29]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [30]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        print('\nTRANSFER')
        contract_utils.parse_contract_execution_json(transfer_passport_json, row=row)
    elif index == 2:
        burn_passport_json = contract_utils.burn_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            display_data=DISPLAY_TX_EXECUTION)
        print('\nBURN')
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    elif index == 3:
        update_name_json = contract_utils.update_name(
            row,
            new_nickname=row['nickname'] + '_new',
            display_data=DISPLAY_TX_EXECUTION)
        print('\nUPDATE NAME')
        contract_utils.parse_contract_execution_json(update_name_json, row=row)
    elif index == 4:
        update_avatar_json = contract_utils.update_avatar(
            row,
            new_avatar=get_ipfs_cid_from_str(row['nickname'] + '_new_avatar'),
            token_id=get_passport_id(row['bostrom_address']),
            display_data=DISPLAY_TX_EXECUTION)
        print('\nUPDATE AVATAR')
        contract_utils.parse_contract_execution_json(update_avatar_json, row=row)
    elif index == 5:
        delete_address_json = contract_utils.delete_address(
            row,
            removed_address=row['cosmos_address'],
            token_id=get_passport_id(row['bostrom_address']),
            display_data=DISPLAY_TX_EXECUTION)
        print('\nDELETE ADDRESS')
        contract_utils.parse_contract_execution_json(delete_address_json, row=row)

0it [00:00, ?it/s]


TRANSFER

Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom1shgxepz7m5dycfmd0gd8vxxvp8uelx6pxfxf0v
	recipient: Passport Owner Address
	token_id: 542447
Gas used: 433,764
Tx hash: 74C08EC0EB7D6898DEB4B27F95528CBBC363AC5D114977C7E5399A1FADD7000C

BURN

Events

cyberlinks
	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

mess

### Add tests for Passport contract
#### create_passport +
#### update_name +
#### update_avatar +
#### proof_address +
#### remove_address +
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +